In [35]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys

sys.path.append('..')
from modeling.xgboost_simple_features import prepare_rolling_window_data

In [61]:
features = pd.read_csv('../data/mood_prediction_simple_features.csv')
features['time'] = pd.to_datetime(features['time'])
#features['date'] = features['time'].dt.date

features.to_csv('features.csv', index=False)

In [ ]:
features

In [28]:
features[['mood','date','id']].groupby(['id','date']).agg('count').reset_index()

,id,date,mood
0,AS14.01,2014-02-17,0
1,AS14.01,2014-02-18,0
2,AS14.01,2014-02-19,0
3,AS14.01,2014-02-20,0
4,AS14.01,2014-02-21,0
...,...,...,...
1968,AS14.33,2014-05-27,5
1969,AS14.33,2014-05-28,5
1970,AS14.33,2014-05-29,3
1971,AS14.33,2014-05-30,5


In [31]:
# For each user, split by time:
grouped = features.groupby('id')

train_dfs = []
test_dfs = []

for user_id, group in grouped:
    group_sorted = group.sort_values('date')  # chronological order
    split_idx = int(len(group_sorted) * 0.8)
    train_dfs.append(group_sorted.iloc[:split_idx])
    test_dfs.append(group_sorted.iloc[split_idx:])

train_data = pd.concat(train_dfs)
test_data = pd.concat(test_dfs)

In [62]:
X, y, dates, user_id = prepare_rolling_window_data(features, window_size=3)

y

X.shape

(1777, 14)

In [63]:
full_df = pd.concat([X,pd.Series(y,name='target_mood')], axis=1)
full_df


full_df[['target_mood','mood_trend']]

,target_mood,mood_trend
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,6.250000,NaN
4,6.333333,NaN
...,...,...
1772,6.200000,-0.40
1773,8.200000,-0.05
1774,7.000000,2.20
1775,6.800000,0.80
